# Задание **Pro**

Попробуйте улучшить текущий скрипт чат-бота, внедрив блок кода для присвоения словам вне словаря (out-of-vocabulary) метки «unknown» так, чтобы, встретив в запросе незнакомое слово, исполнение кода не останавливалось, а продолжалось, игнорируя «unknown» слова.

# Подключение библиотек

In [1]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical, plot_model

from google.colab import files
import numpy as np
import yaml

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Парсинг данных

In [3]:
# Открываем файл с диалогами
corpus = open('/content/drive/MyDrive/УИИ/Генерация текста/Диалоги.yml', 'r')
document = yaml.safe_load(corpus)
conversations = document['разговоры']
print(f'Количество пар вопрос-ответ : {len(conversations)}')
print(f'Пример диалога : {conversations[10000]}')
corpus.close()

Количество пар вопрос-ответ : 11893
Пример диалога : ['Откуда вы знаете?', 'Я  Чорин-Цу,  господин.']


In [4]:
# Разбираем вопросы-ответы с проставлением тегов ответам
questions = []
answers   = []

for con in conversations[:len(conversations)//2]:   # Из-за нехватки памяти пришлось сократить обучающий набор пополам
  if len(con) > 2:
    questions.append(con[0])
    replies = con[1:]
    ans = ''
    for rep in replies:
      ans += " " + rep
    answers.append(ans)
  elif len(con) > 1:
    questions.append(con[0])
    answers.append(con[1])

In [5]:
# Очищаем строки с неопределенным типом ответов
answersCleaned = []
for i in range(len(answers)):
  if type(answers[i]) == str:
    answersCleaned.append(answers[i])
  else:
    questions.pop()

In [6]:
# Сделаем теги-метки для начала и конца ответов
answers = []
for i in range(len(answersCleaned)):
  answers.append('<START>' + answersCleaned[i] + '<END>')

In [7]:
# Выведем обновленные данные на экран
print('Вопрос : {}'.format(questions[5000]))
print('Ответ  : {}'.format(answers[5000]))

Вопрос : Что?..
Ответ  : <START>К Туманной Скале!<END>


## Токенайзер

Для токенизации устанавливаем дополнительный парметр **oov_token='unknown'** для того чтобы все неизвестые слова имели индекс **1** с названием **'unknown'**. Так же как и в уроке работа с текстами писателей. 

In [36]:
# Подключаем керасовский токенизатор и собираем словарь индексов
#tokenizer = Tokenizer()
tokenizer = Tokenizer(oov_token='unknown')
tokenizer.fit_on_texts(questions + answers)
vocabularyItems = list(tokenizer.word_index.items())
vocabularySize = len(vocabularyItems) + 1

In [37]:
print( 'Размер словаря   : {}'.format(vocabularySize))
print( 'Фрагмент словаря : {}'.format(vocabularyItems[:100]))

Размер словаря   : 9383
Фрагмент словаря : [('unknown', 1), ('start', 2), ('end', 3), ('что', 4), ('не', 5), ('а', 6), ('ты', 7), ('я', 8), ('это', 9), ('в', 10), ('как', 11), ('и', 12), ('да', 13), ('нет', 14), ('вы', 15), ('ну', 16), ('на', 17), ('с', 18), ('же', 19), ('где', 20), ('так', 21), ('у', 22), ('кто', 23), ('он', 24), ('то', 25), ('все', 26), ('тебя', 27), ('мы', 28), ('куда', 29), ('мне', 30), ('там', 31), ('есть', 32), ('почему', 33), ('вот', 34), ('за', 35), ('меня', 36), ('тебе', 37), ('ничего', 38), ('здесь', 39), ('еще', 40), ('знаю', 41), ('ли', 42), ('товарищ', 43), ('его', 44), ('к', 45), ('чего', 46), ('вас', 47), ('о', 48), ('надо', 49), ('зачем', 50), ('может', 51), ('вам', 52), ('сейчас', 53), ('по', 54), ('они', 55), ('нас', 56), ('можно', 57), ('чем', 58), ('тут', 59), ('бы', 60), ('но', 61), ('из', 62), ('она', 63), ('тоже', 64), ('конечно', 65), ('какой', 66), ('будет', 67), ('очень', 68), ('случилось', 69), ('уже', 70), ('дело', 71), ('сам', 72), ('скольк

# Подготовка выборки

In [38]:
# Устанавливаем закодированные входные данные(вопросы)
tokenizedQuestions = tokenizer.texts_to_sequences(questions)
maxLenQuestions = max([len(x) for x in tokenizedQuestions])

# Делаем последовательности одной длины
encoderForInput = pad_sequences(tokenizedQuestions, maxlen=maxLenQuestions, padding='post')

In [39]:
print('Пример оригинального вопроса на вход : {}'.format(questions[1000])) 
print('Пример кодированного вопроса на вход : {}'.format(encoderForInput[1000])) 
print('Размеры закодированного массива вопросов на вход : {}'.format(encoderForInput.shape)) 
print('Установленная длина вопросов на вход : {}'.format(maxLenQuestions)) 

Пример оригинального вопроса на вход : Куда теперь?
Пример кодированного вопроса на вход : [29 83  0  0  0  0  0  0  0  0  0]
Размеры закодированного массива вопросов на вход : (5943, 11)
Установленная длина вопросов на вход : 11


In [40]:
# Устанавливаем раскодированные входные данные(ответы)
tokenizedAnswers = tokenizer.texts_to_sequences(answers)
maxLenAnswers = max([len(x) for x in tokenizedAnswers])

# Делаем последовательности одной длины
decoderForInput = pad_sequences(tokenizedAnswers, maxlen=maxLenAnswers, padding='post')

In [13]:
print('Пример оригинального ответа на вход: {}'.format(answers[1000])) 
print('Пример раскодированного ответа на вход : {}'.format(decoderForInput[1000])) 
print('Размеры раскодированного массива ответов на вход : {}'.format(decoderForInput.shape)) 
print('Установленная длина ответов на вход : {}'.format(maxLenAnswers)) 

Пример оригинального ответа на вход: <START>На Родину!<END>
Пример раскодированного ответа на вход : [  2  17 943   3   0   0   0   0   0   0   0   0   0]
Размеры раскодированного массива ответов на вход : (5943, 13)
Установленная длина ответов на вход : 13


In [41]:
# Раскодированные выходные данные(ответы)
for i in range(len(tokenizedAnswers)):
  tokenizedAnswers[i] = tokenizedAnswers[i][1:]  # избавляемся от тега <START>

# Делаем последовательности одной длины
paddedAnswers = pad_sequences(tokenizedAnswers, maxlen=maxLenAnswers, padding='post')

In [42]:
# переводим в one hot vector
oneHotAnswers = to_categorical(paddedAnswers, vocabularySize)
decoderForOutput = np.array(oneHotAnswers)

In [16]:
print('Пример раскодированного ответа на вход : {}'.format(decoderForInput[4999][:21]))  
print('Пример раскодированного ответа на выход : {}'.format(decoderForOutput[4999][4][:21])) 
print('Размеры раскодированного массива ответов на выход : {}'.format(decoderForOutput.shape))
print('Установленная длина вопросов на выход : {}'.format(maxLenAnswers)) 

Пример раскодированного ответа на вход : [   2    8    5 8941  421    3    0    0    0    0    0    0    0]
Пример раскодированного ответа на выход : [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Размеры раскодированного массива ответов на выход : (5943, 13, 9383)
Установленная длина вопросов на выход : 13


In [43]:
# Создаем рабочую модель для вывода ответов на запросы пользователя
def makeInferenceModels():
  encoderModel = Model(encoderInputs, encoderStates)

  decoderStateInput_h = Input(shape=(200,))
  decoderStateInput_c = Input(shape=(200,))
  decoderStatesInputs = [decoderStateInput_h, decoderStateInput_c]

  decoderOutputs, state_h, state_c = decoderLSTM(decoderEmbedding, initial_state=decoderStatesInputs)
  decoderStates = [state_h, state_c]
  decoderOutputs = decoderDense(decoderOutputs)

  decoderModel = Model([decoderInputs] + decoderStatesInputs, [decoderOutputs] + decoderStates)

  return encoderModel, decoderModel

In [44]:
######################
# Создадим функцию, которая преобразует вопрос пользователя в последовательность индексов
######################
def strToTokens(sentence):      # функция принимает строку на вход (предложение с вопросом)
  CheckIndex = tokenizer.texts_to_sequences([sentence])
  return pad_sequences(CheckIndex, maxlen=maxLenQuestions , padding='post')

In [67]:
# Устанавливаем окончательные настройки и запускаем модель 
def TestModel():
  encModel , decModel = makeInferenceModels()

  for _ in range(5):
    statesValues = encModel.predict(pad_sequences(tokenizer.texts_to_sequences([input('Задайте вопрос : ')]), maxlen=maxLenQuestions , padding='post'))
    emptyTargetSeq = np.zeros((1, 1))    
    emptyTargetSeq[0, 0] = tokenizer.word_index['start']

    stopCondition = False
    decodedTranslation = ''
    while not stopCondition :
      decOutputs , h , c = decModel.predict([emptyTargetSeq] + statesValues)
      sampledWordIndex = np.argmax( decOutputs[0, 0, :])
      sampledWord = None
      for word , index in tokenizer.word_index.items():
        if sampledWordIndex == index:
          decodedTranslation += ' {}'.format(word)
          sampledWord = word
      
      if sampledWord == 'end' or len(decodedTranslation.split()) > maxLenAnswers:
        stopCondition = True

      emptyTargetSeq[0, 0] = sampledWordIndex
      statesValues = [h, c]
    
    print(decodedTranslation[:-3])

# Параметры нейросети и модель обучения

In [46]:
# Первый входной слой, кодер, выходной слой
encoderInputs = Input(shape=(None, ))
encoderEmbedding = Embedding(vocabularySize, 200, mask_zero=True)(encoderInputs)
encoderOutputs, state_h, state_c = LSTM(200, return_state=True)(encoderEmbedding)
encoderStates = [state_h, state_c]

In [47]:
# Второй входной слой, декодер, выходной слой
decoderInputs = Input(shape=(None, ))
decoderEmbedding = Embedding(vocabularySize, 200, mask_zero=True)(decoderInputs)
decoderLSTM = LSTM(200, return_state=True, return_sequences=True)
decoderOutputs, _ , _ = decoderLSTM (decoderEmbedding, initial_state=encoderStates)
decoderDense =  Dense(vocabularySize, activation='softmax')
output = decoderDense(decoderOutputs)

In [48]:
# Собираем тренировочную модель нейросети
model = Model([encoderInputs, decoderInputs], output)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy')

## Обучение 20 эпох

In [56]:
model.fit([encoderForInput , decoderForInput], decoderForOutput, batch_size=50, epochs=50)

Epoch 1/50
119/119 [==============================] - 4s 33ms/step - loss: 1.3218
Epoch 2/50
119/119 [==============================] - 4s 34ms/step - loss: 1.2903
Epoch 3/50
119/119 [==============================] - 5s 39ms/step - loss: 1.2603
Epoch 4/50
119/119 [==============================] - 4s 32ms/step - loss: 1.2311
Epoch 5/50
119/119 [==============================] - 4s 33ms/step - loss: 1.2020
Epoch 6/50
119/119 [==============================] - 4s 32ms/step - loss: 1.1747
Epoch 7/50
119/119 [==============================] - 4s 32ms/step - loss: 1.1456
Epoch 8/50
119/119 [==============================] - 4s 33ms/step - loss: 1.1179
Epoch 9/50
119/119 [==============================] - 4s 36ms/step - loss: 1.0911
Epoch 10/50
119/119 [==============================] - 4s 32ms/step - loss: 1.0617
Epoch 11/50
119/119 [==============================] - 4s 33ms/step - loss: 1.0344
Epoch 12/50
119/119 [==============================] - 4s 34ms/step - loss: 1.0083
Epoch 13/50
1

In [66]:
# Сохранение весов
model.save_weights('/content/drive/MyDrive/УИИ/Генерация текста/Pro.h5')

## Проверка результата


In [ ]:
# Загрузка весов
#model.load_weights('/content/drive/MyDrive/УИИ/Генерация текста/Pro.h5')

In [68]:
TestModel()

Задайте вопрос : Как твои дела
 все в порядке 
Задайте вопрос : Есть ли жизнь на Марсе
 нет в порядке 
Задайте вопрос : ку
 нет ничего 
Задайте вопрос : А что если авыаывроавы
 это тебе теперь 
Задайте вопрос : А вот если так оыводлыап
 мне бы и хорошо 


# Выводы:

1. Обучение происходило лишь на половине примеров из-за нехватки памяти. Но в целом ответы нейронной сети приемлемы.
2. Слова которые не входят в ее словарь принимают значения **unknow** с индексом **1**. Что не вызывает ошибок.
3. Перевод в цифровое значение происходит так же как и в занятиях по работе с текстами писателей. 
4. Данный пример показывает что возможно хорошо обучить нейронную сеть имея большее кол-во разнообразных примеров. Пробуя различную архитектуру и оптимальный подбор гиперпараметров.  